In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
!cp /content/drive/MyDrive/foodimages.zip /content/


In [4]:
import zipfile

with zipfile.ZipFile('/content/foodimages.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/')


In [5]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

In [6]:
# Paths
BASE_DIR = "/content/foodimages"
TRAIN_DIR = os.path.join(BASE_DIR, "train")
VAL_DIR = os.path.join(BASE_DIR, "validation")
TEST_DIR = os.path.join(BASE_DIR, "test")
CHECKPOINT_DIR = "/content/checkpoints"
os.makedirs(CHECKPOINT_DIR, exist_ok=True)
LATEST_CHECKPOINT = tf.train.latest_checkpoint(CHECKPOINT_DIR)

In [7]:
# Parameters
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
NUM_CLASSES = 101
TOTAL_EPOCHS = 60

In [8]:
# Data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    shear_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.7, 1.3],
    fill_mode='nearest'
)

In [9]:
val_test_datagen = ImageDataGenerator(rescale=1./255)

In [10]:
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

Found 35350 images belonging to 101 classes.


In [11]:
val_generator = val_test_datagen.flow_from_directory(
    VAL_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

Found 17776 images belonging to 101 classes.


In [12]:
test_generator = val_test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

Found 17776 images belonging to 101 classes.


In [13]:
# Build MobileNetV2
base_model = MobileNetV2(include_top=False, input_shape=(224, 224, 3), weights='imagenet')
base_model.trainable = True  # Fine-tuning from beginning

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [14]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.4)(x)
outputs = Dense(NUM_CLASSES, activation='softmax')(x)

In [15]:
model = Model(inputs=base_model.input, outputs=outputs)

In [16]:
# Compile model
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [17]:
# If checkpoint exists, load it
initial_epoch = 0
if LATEST_CHECKPOINT:
    print(f"✅ Found checkpoint {LATEST_CHECKPOINT}, loading weights...")
    model.load_weights(LATEST_CHECKPOINT)
    # Extract epoch number from checkpoint filename
    initial_epoch = int(LATEST_CHECKPOINT.split('_')[-1].split('.')[0])
    print(f"Resuming from epoch {initial_epoch}...")

else:
    print("🚀 No checkpoint found, starting fresh...")

🚀 No checkpoint found, starting fresh...


In [18]:
# Save checkpoint after every epoch
checkpoint_cb = ModelCheckpoint(
    filepath=os.path.join(CHECKPOINT_DIR, "ckpt_{epoch:02d}.weights.h5"),
    save_weights_only=True,
    save_freq='epoch'
)

In [19]:
# Train model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=TOTAL_EPOCHS,
    initial_epoch=initial_epoch,
    callbacks=[checkpoint_cb]
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/60
1105/1105 ━━━━━━━━━━━━━━━━━━━━ 611s 499ms/step - accuracy: 0.1826 - loss: 3.7366 - val_accuracy: 0.4309 - val_loss: 2.3339
Epoch 2/60
1105/1105 ━━━━━━━━━━━━━━━━━━━━ 514s 465ms/step - accuracy: 0.5224 - loss: 1.8771 - val_accuracy: 0.5955 - val_loss: 1.5797
Epoch 3/60
1105/1105 ━━━━━━━━━━━━━━━━━━━━ 511s 462ms/step - accuracy: 0.5992 - loss: 1.5277 - val_accuracy: 0.6130 - val_loss: 1.5203
Epoch 4/60
1105/1105 ━━━━━━━━━━━━━━━━━━━━ 512s 464ms/step - accuracy: 0.6481 - loss: 1.3251 - val_accuracy: 0.6487 - val_loss: 1.3812
Epoch 5/60
1105/1105 ━━━━━━━━━━━━━━━━━━━━ 512s 464ms/step - accuracy: 0.6889 - loss: 1.1555 - val_accuracy: 0.6147 - val_loss: 1.5674
Epoch 6/60
1105/1105 ━━━━━━━━━━━━━━━━━━━━ 522s 472ms/step - accuracy: 0.7198 - loss: 1.0223 - val_accuracy: 0.6497 - val_loss: 1.3979
Epoch 7/60
1105/1105 ━━━━━━━━━━━━━━━━━━━━ 513s 464ms/step - accuracy: 0.7410 - loss: 0.9501 - val_accuracy: 0.6603 - val_loss: 1.3527
Epoch 8/60
1105/1105 ━━━━━━━━━━━━━━━━━━━━ 510s 462ms/step - ac

In [20]:
# Rebuild MobileNetV2 model architecture
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

In [21]:
IMG_SIZE = (224, 224)
NUM_CLASSES = 101

In [22]:
# Build the model again
base_model = MobileNetV2(include_top=False, input_shape=(224, 224, 3), weights='imagenet')
base_model.trainable = True

In [23]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.4)(x)
outputs = Dense(NUM_CLASSES, activation='softmax')(x)


In [24]:
model = Model(inputs=base_model.input, outputs=outputs)

In [25]:
# Compile model
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [26]:
# Load your best checkpoint
best_checkpoint_path = '/content/checkpoints/ckpt_57.weights.h5'
model.load_weights(best_checkpoint_path)
print("✅ Successfully loaded best checkpoint from epoch 57!")

✅ Successfully loaded best checkpoint from epoch 57!


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 318 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [27]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_generator)
print(f"\n✅ Final Test Accuracy from best checkpoint: {test_acc:.4f}")

556/556 ━━━━━━━━━━━━━━━━━━━━ 49s 80ms/step - accuracy: 0.6682 - loss: 2.1094

✅ Final Test Accuracy from best checkpoint: 0.6904


In [29]:
# Save the final good model
model.save('/content/mobilenetv2_best_model_epoch57.h5')
print("\n✅ Final best model saved successfully!")


✅ Final best model saved successfully!
